In [1]:
"""
Multi-Agent Dynamic Grid World Environment
Created by: Ardianto Wibowo
"""

import numpy as np
import sys

# Add the path to the 'env' folder to sys.path
sys.path.append('env')

from ma_gridworld import Env

class SearchAgent:
    def __init__(self, num_actions):
        self.num_actions = num_actions
        self.targets_seen = []
        self.memory = []  


  


    def analyse_sensor_data(self, agent_id, coordinate_observation, sensor_data_observation):
        for i in range(len(sensor_data_observation)):
            for j in range(len(sensor_data_observation[i])):
                data = sensor_data_observation[i][j]
                location = [coordinate_observation[0] + j - len(sensor_data_observation[i])//2,
                           coordinate_observation[1] + i - len(sensor_data_observation)//2]
                if data != None and 'target_' + str(agent_id) in data:
                    if location not in self.targets_seen:
                        self.targets_seen.append(location)
                    # تحديث الذاكرة مع تمرير agent_id
                if data is not None and 'target_' in data:  # التحقق من وجود هدف
                    self.update_memory(location, data, agent_id)  # استدعاء update_memory

                # إزالة الأهداف التي تم جمعها
                if location[0] == coordinate_observation[0] and location[1] == coordinate_observation[1]:
                    if location in self.targets_seen:
                        self.targets_seen.remove(location)








        
    def analyse_communication(self, comm_observation, agent_id):
        for comm in comm_observation:
            origin_location = comm[0]
            sensor_data_observation = comm[1]
            reported_by = comm[2]

            # طباعة البيانات المستلمة للتحقق
         

            for i in range(len(sensor_data_observation)):
                for j in range(len(sensor_data_observation[i])):
                    data = sensor_data_observation[i][j]
                    location = [origin_location[0] + j - len(sensor_data_observation[i])//2,
                               origin_location[1] + i - len(sensor_data_observation)//2]
                    if data != None and 'target_' + str(agent_id) in data:
                        if location not in self.targets_seen:
                            self.targets_seen.append(location)
                            print(f"Agent {agent_id}: Add target to {location} sent by Agent {reported_by}")                   

                           
                            
    def update_memory(self, location, data, agent_id):

        print(f"Agent {agent_id} is updating memory for location {location} with data {data}")
        if location not in [record["location"] for record in self.memory]:
            # add
            self.memory.append({"location": location, "data": data})
            print(f"Agent {agent_id} Updated Memory: {location} -> {data}")





    def select_action(self, coordinate_observation):
        if len(self.targets_seen) > 0:
            # find the closest target
            closest_target = None
            closest_target_distance = 999999
            for target_coordinate in self.targets_seen:
                horizontal_distance = target_coordinate[0] - coordinate_observation[0]
                vertical_distance = target_coordinate[1] - coordinate_observation[1]
                distance = abs(horizontal_distance) + abs(vertical_distance)
                if distance < closest_target_distance:
                    closest_target_distance = distance
                    closest_target = target_coordinate
            horizontal_distance = closest_target[0] - coordinate_observation[0]
            vertical_distance = closest_target[1] - coordinate_observation[1]
            if abs(horizontal_distance) >= abs(vertical_distance):
                if horizontal_distance < 0:
                    return 3
                elif horizontal_distance > 0:
                    return 4
                return 0
            else:
                if vertical_distance < 0:
                    return 1
                elif vertical_distance > 0:
                    return 2
                return 0
        else:
            return np.random.choice(num_actions-1) + 1 # example of random value as a physical action


def get_action(agent_id, observation, num_actions, agents, env):
    """
    This method provides a random action chosen recognized by the ma-gridworld environment:
    1: up, 2: down, 3: left, 4: right, 0: stay
    """
    
    coordinate_observation = tuple(observation[0])  # Keep observation as (x, y) tuple

    # Optional observation data may be used, depend on the agent needs.
    win_state_observation = observation[1]
    sensor_data_observation = observation[2]
    comm_observation = observation[3]
    

    print(f"Observation for Agent {agent_id}: {observation}")

    # Check if communication data is available
    if comm_observation:
        agents[agent_id].analyse_communication(comm_observation, agent_id)
    else:
        print(f"Agent {agent_id}: No communication data available this step.")

    # Analyze sensor data
    agents[agent_id].analyse_sensor_data(agent_id, coordinate_observation, sensor_data_observation)
    
    # Select physical action
    physical_action = agents[agent_id].select_action(coordinate_observation)

    # Define communication action
    if env.is_agent_silent:
        comm_action = []  # Communication action is set to be zero if agent silent
    else:
        comm_action = [coordinate_observation, sensor_data_observation, agent_id]  # Add agent_id as 'reported_by'
    
    return (physical_action, comm_action)


def run(num_episodes, max_steps_per_episode, agents, num_actions, env):
    for episode in range(num_episodes):
        print(f"Starting episode {episode + 1}")
        observations = env.reset()  # Reset the environment at the start of each episode
        
        for agent in agents:
            agent.__init__(num_actions)
        
        done = [False] * env.num_agents  # Initialize 'done' as a list for each agent
        step_count = 0

        with open("simulation_results.txt", "a") as file:  # فتح ملف النص لتخزين النتائج
            file.write(f"Starting episode {episode + 1}\n")

            while not all(done) and step_count < max_steps_per_episode:  # Stop if all agents are done or max steps reached
                actions = []
                next_observations = []
                
                for agent_id in range(env.num_agents):
                    observation = observations[agent_id]
                    action = get_action(agent_id, observation, num_actions, agents, env)
                    
                    actions.append(action)
                    next_observations.append(observation)

                next_observations, rewards, done = env.step(actions)  # Step in the environment

                observations = next_observations
                step_count += 1

                # Render the environment
                env.render()

                # كتابة النتائج في الملف النصي
                file.write(f"Step {step_count}:\n")
                for agent_id in range(env.num_agents):
                    file.write(
                        f"  Agent {agent_id}: Observation: {observations}, "
                        f"Action: {actions[agent_id]}, Reward: {rewards[agent_id]}, Done: {done[agent_id]}\n"
                    )

            file.write(f"Episode {episode + 1} finished after {step_count} steps.\n\n")




if __name__ == "__main__":

    gsize=15 #grid size (square)
    gpixels=30 #grid cell size in pixels

    is_sensor_active = True #True:  Activate the sensory observation data
    sensory_size = 3 #'is_sensor_active' must be True. The value must be odd, if event will be converted to one level odd number above
    
    num_agents = 10 #the number of agents will be run in paralel
    num_obstacles = 0 #the number of obstacles
    is_single_target = False #True: all agents have a single target, False: each agent has their own target
    num_targets_per_agent = 5 #'is_single_target' must be true to have an effect
    
    is_agent_silent = False #True: communication among agents is allowed

    num_episodes=1 #the number of episode will be run
    max_steps_per_episode=1000 #each episode will be stopped when max_step is reached

    eps_moving_targets = 10 #set this value greater than 'num_episodes' to keep the targets in a stationary position
    eps_moving_obstacles = 10 #set this value greater than 'num_episodes' to keep the obstacles in a stationary position

    render = True #True: render the animation into the screen (so far, it is still can not be deactivated)

    min_obstacle_distance_from_target = 1 #min grid distance of each obstacles relative to targets
    max_obstacle_distance_from_target = 5 #max grid distance of each obstacles relative to targets
    min_obstacle_distance_from_agents = 1 #min grid distance of each obstacles relative to agents

    reward_normal = -1 #reward value of normal steps
    reward_obstacle = -5 #reward value when hit an obstacle
    reward_target = 50 #reward value when reach the target

    is_totally_random = True #True: target and obstacles initial as well as movement position is always random on each call, False: only random at the beginning. 
    animation_speed = 0.1 #smaller is faster 
    is_destroy_environment = True #True: automatically close the animation after all episodes end.  

    # Initialize environment
    env = Env(
        num_agents=num_agents, num_targets_per_agent=num_targets_per_agent, num_obstacles=num_obstacles,
        eps_moving_obstacles=eps_moving_obstacles, eps_moving_targets=eps_moving_targets,
        is_agent_silent=is_agent_silent, is_single_target=is_single_target, sensory_size=sensory_size,
        gpixels=gpixels, gheight=gsize, gwidth=gsize, is_sensor_active=is_sensor_active,
        min_obstacle_distance_from_target=min_obstacle_distance_from_target,
        max_obstacle_distance_from_target=max_obstacle_distance_from_target,
        min_obstacle_distance_from_agents=min_obstacle_distance_from_agents,
        is_totally_random=is_totally_random, animation_speed=animation_speed,
        reward_normal=reward_normal, reward_obstacle=reward_obstacle, reward_target=reward_target
    )
    
    num_actions = len(env.action_space)
    
    # Initialize Q-learning agents
    agents = [SearchAgent(num_actions) for _ in range(num_agents)]

    # Run episodes
    run(num_episodes, max_steps_per_episode, agents, num_actions, env)

    if is_destroy_environment:
        env.destroy_environment() 


Starting episode 1
Observation for Agent 0: [[0, 0], False, [[None, None, None], [None, 'agent', 'empty'], [None, 'empty', 'empty']], []]
Agent 0: No communication data available this step.
Observation for Agent 1: [[14, 0], False, [[None, None, None], ['empty', 'agent', None], ['empty', 'empty', None]], []]
Agent 1: No communication data available this step.
Observation for Agent 2: [[14, 14], False, [['empty', 'empty', None], ['empty', 'agent', None], [None, None, None]], []]
Agent 2: No communication data available this step.
Observation for Agent 3: [[0, 14], False, [[None, 'empty', 'empty'], [None, 'agent', 'empty'], [None, None, None]], []]
Agent 3: No communication data available this step.
Observation for Agent 4: [[5, 0], False, [[None, None, None], ['empty', 'agent', 'empty'], ['target_6_4', 'empty', 'target_3_2']], []]
Agent 4: No communication data available this step.
Agent 4 is updating memory for location [4, 1] with data target_6_4
Agent 4 Updated Memory: [4, 1] -> targ

Observation for Agent 0: [[0, 0], False, [[None, None, None], [None, 'agent', 'empty'], [None, 'empty', 'empty']], [[(14, 0), [[None, None, None], ['empty', 'agent', None], ['empty', 'empty', None]], 1], [(13, 14), [['target_9_2', 'empty', 'empty'], ['empty', 'empty', 'agent'], [None, None, None]], 2], [(0, 13), [[None, 'empty', 'empty'], [None, 'empty', 'empty'], [None, 'agent', 'empty']], 3], [(6, 0), [[None, None, None], ['agent', 'empty', 'empty'], ['empty', 'target_3_2', 'empty']], 4], [(8, 0), [[None, None, None], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 5], [(14, 5), [['target_8_0', 'empty', None], ['empty', 'agent', None], ['empty', 'empty', None]], 6], [(14, 8), [['target_5_1', 'empty', None], ['target_5_4', 'empty', None], ['empty', 'agent', None]], 7], [(6, 14), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], [None, None, None]], 8], [(0, 6), [[None, 'empty', 'target_7_4'], [None, 'empty', 'empty'], [None, 'agent', 'empty']], 9]]]
Observation for

Observation for Agent 0: [[0, 0], False, [[None, None, None], [None, 'agent', 'empty'], [None, 'empty', 'empty']], [[(14, 2), [['empty', 'agent', None], ['empty', 'empty', None], ['target_1_3', 'empty', None]], 1], [(11, 14), [['empty', 'target_7_2', 'target_9_2'], ['empty', 'empty', 'agent'], [None, None, None]], 2], [(0, 11), [[None, 'empty', 'empty'], [None, 'empty', 'empty'], [None, 'agent', 'empty']], 3], [(6, 1), [['empty', 'agent', 'empty'], ['empty', 'target_3_2', 'empty'], ['empty', 'empty', 'empty']], 4], [(8, 2), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 5], [(12, 5), [['empty', 'target_6_3', 'target_8_0'], ['target_1_1', 'empty', 'agent'], ['empty', 'empty', 'empty']], 6], [(13, 7), [['empty', 'empty', 'empty'], ['empty', 'target_5_1', 'agent'], ['empty', 'target_5_4', 'empty']], 7], [(6, 12), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 8], [(1, 6), [['empty', 'target_7_4', 'empty'],

Observation for Agent 0: [[0, 1], False, [[None, 'empty', 'empty'], [None, 'agent', 'empty'], [None, 'empty', 'empty']], [[(13, 3), [['target_5_3', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['agent', 'target_8_0', 'empty']], 1], [(12, 13), [['empty', 'empty', 'empty'], ['target_7_2', 'target_9_2', 'empty'], ['empty', 'agent', 'empty']], 2], [(0, 9), [[None, 'empty', 'empty'], [None, 'empty', 'empty'], [None, 'agent', 'empty']], 3], [(6, 1), [['empty', 'agent', 'empty'], ['empty', 'target_3_2', 'empty'], ['empty', 'empty', 'empty']], 4], [(10, 2), [['empty', 'empty', 'target_4_0'], ['agent', 'empty', 'empty'], ['empty', 'target_5_0', 'target_4_4']], 5], [(11, 4), [['target_5_0', 'target_4_4', 'empty'], ['target_8_3', 'empty', 'agent'], ['empty', 'target_1_1', 'empty']], 6], [(13, 9), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 7], [(7, 11), [['target_4_1', 'target_1_0', 'empty'], ['agent', 'empty', 'target_9_0'], ['empty', 'empty', 'emp

Observation for Agent 0: [[0, 1], False, [[None, 'agent', 'empty'], [None, 'empty', 'empty'], [None, 'empty', 'empty']], [[(12, 4), [['target_4_4', 'agent', 'empty'], ['empty', 'empty', 'target_8_0'], ['target_1_1', 'empty', 'empty']], 1], [(12, 13), [['empty', 'empty', 'empty'], ['target_7_2', 'target_9_2', 'empty'], ['empty', 'agent', 'empty']], 2], [(1, 8), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 3], [(8, 1), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 4], [(11, 3), [['empty', 'empty', 'target_5_2'], ['agent', 'target_4_4', 'empty'], ['agent', 'empty', 'agent']], 5], [(9, 4), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['target_8_2', 'target_0_1', 'empty']], 6], [(13, 11), [['empty', 'agent', 'empty'], ['target_3_4', 'empty', 'empty'], ['empty', 'empty', 'empty']], 7], [(7, 9), [['empty', 'empty', 'empty'], ['empty', 'empty', 'target_2_3'], ['target_4_1', 'agent', 'empty']], 

...

Observation for Agent 0: [[13, 13], False, [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], [[(7, 5), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 1], [(9, 12), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 2], [(5, 10), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'agent']], 3], [(1, 10), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 4], [(8, 9), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 5], [(10, 3), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['agent', 'empty', 'empty']], 6], [(5, 11), [['empty', 'agent', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 7], [(9, 5), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 8], [(1, 4), [['empty', 'empty', 'empty'], ['empty

Observation for Agent 0: [[14, 14], False, [['empty', 'agent', None], ['empty', 'empty', None], [None, None, None]], [[(6, 4), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 1], [(10, 11), [['empty', 'empty', 'target_2_0'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 2], [(4, 11), [['empty', 'agent', 'agent'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 3], [(1, 8), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 4], [(7, 10), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 5], [(9, 2), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 6], [(5, 9), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 7], [(9, 5), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 8], [(2, 5), [['empty', 'agent', 'empty'], ['empty', 'empty',

Observation for Agent 0: [[14, 13], False, [['empty', 'empty', None], ['empty', 'empty', None], ['empty', 'agent', None]], [[(6, 4), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 1], [(11, 10), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 2], [(3, 12), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['target_3_0', 'empty', 'target_5_0']], 3], [(1, 6), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 4], [(6, 11), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 5], [(10, 3), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 6], [(7, 9), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 7], [(8, 6), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 8], [(2, 5), [['empty', 'empty', 'empty'], ['empt

Observation for Agent 0: [[14, 11], False, [['empty', 'empty', None], ['empty', 'empty', None], ['empty', 'agent', None]], [[(6, 4), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 1], [(11, 10), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 2], [(2, 13), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 3], [(1, 8), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 4], [(5, 12), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['target_5_0', 'empty', 'empty']], 5], [(8, 3), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 6], [(8, 8), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 7], [(8, 4), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 8], [(3, 4), [['empty', 'empty', 'empty'], ['agent', 'e

Observation for Agent 0: [[14, 13], True, [['empty', 'agent', None], ['empty', 'empty', None], ['empty', 'empty', None]], [[(8, 4), [['empty', 'empty', 'agent'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 1], [(10, 9), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 2], [(2, 13), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 3], [(2, 7), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 4], [(4, 13), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 5], [(10, 3), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 6], [(9, 9), [['empty', 'empty', 'empty'], ['agent', 'empty', 'agent'], ['empty', 'empty', 'empty']], 7], [(8, 4), [['empty', 'empty', 'empty'], ['agent', 'agent', 'empty'], ['empty', 'empty', 'empty']], 8], [(3, 4), [['empty', 'empty', 'empty'], ['empty', 'empty',